In [59]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/Extra_data.csv'
#df = pd.read_csv(url, sep=";", low_memory =False)

url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [30]:
df

,Year,Company code,FAR total,TRIR total,TRIR company only,TRIR contractor only,LTIR total,LTIR company only,LTIR contractor only
486,2014,ZZ,0,"3,61",0,"5,43",0,0,0
456,2014,W,"2,6",NaN,NaN,NaN,"0,37","0,37","0,37"
455,2014,V,0,"1,41","0,54","1,65","0,39","0,11","0,47"
454,2014,U,"7,03","1,97","1,21","2,2","0,49","0,61","0,46"
453,2014,T,0,"4,47","2,57","6,24","0,5","0,51","0,48"
...,...,...,...,...,...,...,...,...,...
31,2023,GG,"1,06","0,38","0,8","0,32","0,18","0,51","0,13"
30,2023,FF,"1,34","0,98","0,75","1,07","0,18","0,15","0,19"
29,2023,EE,0,"0,46","0,33","0,51","0,18","0,33","0,13"
27,2023,CC,0,"0,2",0,"0,24","0,2",0,"0,24"


In [60]:
df = df.sort_values(by="Year", ascending=True)
#Drop FAR
df.drop("FAR total", axis = 1, inplace=True)

cols = ["TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only" ]  # List of columns to convert
df[cols] = df[cols].replace(',', '.', regex=True).astype(float)

df[cols] = df[cols].astype(float)

df = df.fillna(0)

**EDA**

In [55]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

(487, 8)
Index(['Year', 'Company code', 'TRIR total', 'TRIR company only',
       'TRIR contractor only', 'LTIR total', 'LTIR company only',
       'LTIR contractor only'],
      dtype='object')
Year                      int64
Company code             object
TRIR total              float64
TRIR company only       float64
TRIR contractor only    float64
LTIR total              float64
LTIR company only       float64
LTIR contractor only    float64
dtype: object
Year                     10
Company code             59
TRIR total              247
TRIR company only       188
TRIR contractor only    279
LTIR total              121
LTIR company only       105
LTIR contractor only    138
dtype: int64
Year                    0
Company code            0
TRIR total              0
TRIR company only       0
TRIR contractor only    0
LTIR total              0
LTIR company only       0
LTIR contractor only    0
dtype: int64


In [62]:
df_total = df[["Year", "Company code", "TRIR total", "LTIR total"]]
df_company = df[["Year", "Company code", "TRIR company only", "LTIR company only"]]
df_contractor = df[["Year", "Company code", "TRIR contractor only", "LTIR contractor only"]]

**Time Serie for TRIR**

*ARIMA*

In [18]:
pip install pmdarima --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/625.1 kB ? eta -:--:--
    --------------------------------------- 10.2/625.1 kB ? eta -:--:--
   -- ------------------------------------ 41.0/625.1 kB 653.6 kB/s eta 0:00:01
   ------------- -------------------------- 204.8/625.1 kB 1.8 MB/s eta 0:00:01
   -------------------------------- ------- 501.8/625.1 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 625.1/625.1 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.4/2.8 MB 12.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.3/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.8 MB 10.5 MB/s eta 0:00:01
   ------------------------------- -------- 2.2/2.8 MB 10.8 MB/s eta 0:00:01
   -------------------------------------- - 2.7/2.8 MB 10.7 MB/s eta 0:00:01
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [104]:
company_counts = df_total.groupby("Company code")["Year"].count()
# Get companies with 5 or more rows
valid_companies = company_counts[company_counts >= 5].index

# Filter the original DataFrame to keep only those companies
df_total_filtered = df_total[df_total["Company code"].isin(valid_companies)]
df_total_filtered = df_total_filtered.set_index("Year")

In [72]:
df_total_filtered.nunique()

Year             10
Company code     50
TRIR total      243
LTIR total      121
dtype: int64

In [74]:
df_total_filtered.dtypes

Year              int64
Company code     object
TRIR total      float64
LTIR total      float64
dtype: object

In [107]:
from joblib import Parallel, delayed
import pmdarima as pm
import pandas as pd

# Function to fit ARIMA and return model summary (for Company only)
def fit_arima(group):
    company = group["Company code"].iloc[0]  # Extract the company name
    ts = group["TRIR total"]  # Convert to time series using the 'TRIR total' column


    try:
        # Fit ARIMA model
        model = pm.auto_arima(ts, start_p=1, start_q=1, max_p=3, max_q=3, d=None, 
                              test="adf", seasonal=False, trace=False, error_action="ignore", 
                              suppress_warnings=True, stepwise=True)
        
        print(f"Successfully fitted ARIMA for {company}")  # Add a print statement to verify success
        # Generate predictions and residuals
        preds = model.predict_in_sample()  # In-sample predictions
        residuals = ts.values - preds  # Residuals (actual - predicted)
        return company, model, preds, residuals

    except Exception as e:
        print(f"Error fitting ARIMA for {company}: {str(e)}")
        return company, None, None, None  # Handle failure gracefully


# Debugging the fitting process and checking the data
#print(df_total_filtered.groupby("Company code")["Year"].count())  # Number of rows per company


In [109]:
# Fit ARIMA Models for each company group
(df_total_filtered["Company code"].unique())



array(['W         ', 'V         ', 'U         ', 'T         ',
       'S         ', 'R         ', 'Q         ', 'P         ',
       'O         ', 'N         ', 'M         ', 'L         ',
       'K         ', 'J         ', 'I         ', 'H         ',
       'G         ', 'F         ', 'E         ', 'D         ',
       'C         ', 'B         ', 'A         ', 'Overall   ',
       'X         ', 'AA        ', 'Y         ', 'WW        ',
       'VV        ', 'UU        ', 'TT        ', 'SS        ',
       'RR        ', 'QQ        ', 'PP        ', 'OO        ',
       'NN        ', 'MM        ', 'KK        ', 'LL        ',
       'Z         ', 'CC        ', 'DD        ', 'EE        ',
       'BB        ', 'GG        ', 'HH        ', 'II        ',
       'JJ        ', 'FF        '], dtype=object)

In [108]:
# Fit ARIMA Models for each company group
models = Parallel(n_jobs=-1)(delayed(fit_arima)(group) for _, group in df_total_filtered.groupby("Company code"))
#models_dict = {company_code: model for company_code, model in zip(models, list(df_total_filtered["Company code"].unique()))}
# Check if models are being fitted correctly
#rint(models_dict)

final_df = pd.DataFrame()


for i, company_code in enumerate(df_total_filtered["Company code"].unique()):
    company_df = df_total_filtered[df_total_filtered["Company code"] == company_code]
    preds = models[i][2]
    residuals = models[i][3]
    final_df_rows = pd.concat([company_df, preds, residuals], axis=1, ignore_index=False)
    print(final_df_rows)
    final_df = pd.concat([final_df, final_df_rows], axis=0)



     Company code  TRIR total  LTIR total  predicted_mean  predicted_mean
Year                                                                     
2014   W                 0.00        0.37        0.000000        4.340000
2015   W                 1.41        0.38        6.510021       -2.450021
2016   W                 1.17        0.28        3.780000       -1.330000
2017   W                 0.76        0.25        0.840000        4.820000
2018   W                 3.05        0.28        8.870000       -6.220000
2019   W                 1.78        0.32       -0.360000        3.610000
2020   W                 0.83        0.25        3.850000        2.800000
2021   W                 0.81        0.23       10.050000       -7.550000
2022   W                 0.52        0.26       -1.650000        3.600000
2023   W                 0.74        0.30        1.400000        1.170000
     Company code  TRIR total  LTIR total  predicted_mean  predicted_mean
Year                                  

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [94]:
print(models[0])

('A         ', ARIMA(order=(0, 2, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False), Year
2014     0.000000
2015     6.510021
2016     3.780000
2017     0.840000
2018     8.870000
2019    -0.360000
2020     3.850000
2021    10.050000
2022    -1.650000
2023     1.400000
Name: predicted_mean, dtype: float64, Year
2014    4.340000
2015   -2.450021
2016   -1.330000
2017    4.820000
2018   -6.220000
2019    3.610000
2020    2.800000
2021   -7.550000
2022    3.600000
2023    1.170000
Name: predicted_mean, dtype: float64)


In [93]:
# Merge the ARIMA features back with the original DataFrame
df_time_with_arima = df_total_filtered.merge(
    arima_features_df[["Company code", "Year", "TRIR_ARIMA_Pred", "TRIR_ARIMA_Residual"]],
    on=["Company code", "Year"], how="left"
)

# Drop rows with missing ARIMA predictions (if ARIMA failed for some companies)
#df_time_with_arima = df_time_with_arima.dropna()

# Display the DataFrame with ARIMA features
print(df_time_with_arima.head())

   Year Company code  TRIR total  LTIR total  TRIR_ARIMA_Pred  \
0  2014   W                 0.00        0.37              NaN   
1  2014   V                 1.41        0.39              NaN   
2  2014   U                 1.97        0.49              NaN   
3  2014   T                 4.47        0.50              NaN   
4  2014   S                 1.29        0.53              NaN   

   TRIR_ARIMA_Residual  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN  


***LTIR Prediction***

**Train test split**

In [ ]:
# Features for model stacking: TRIR ARIMA predictions and residuals
features = ['TRIR_ARIMA_Pred', 'TRIR_ARIMA_Residual',
            'TRIR company only_ARIMA_Pred', 'TRIR company only_ARIMA_Residual',
            'TRIR contractor only_ARIMA_Pred', 'TRIR contractor only_ARIMA_Residual']

# Prepare the data for Random Forest model stacking
X = df_time_with_arima[features]
y_total = df_time_with_arima['LTIR total']
y_company = df_time_with_arima['LTIR company only']
y_contractor = df_time_with_arima['LTIR contractor only']

# Split the data into training and testing sets
X_train, X_test, y_train_total, y_test_total = train_test_split(X, y_total, test_size=0.2, random_state=42)
X_train, X_test, y_train_company, y_test_company = train_test_split(X, y_company, test_size=0.2, random_state=42)
X_train, X_test, y_train_contractor, y_test_contractor = train_test_split(X, y_contractor, test_size=0.2, random_state=42)

**Model selection and evaluation only Model stacking for LTIR total, company and global**



*Random forest*

In [ ]:
#Class model
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=10, random_state=42)
forest.fit(X_train, y_train_total)
pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test_total))
print("RMSE", mean_squared_error(pred, y_test_total, squared=False))
print("R2 score (Test)", forest.score(X_test, y_test_total))
print("R2 score (Train)", forest.score(X_train, y_train_total))

MAE 0.10721287031055741
RMSE 0.20366342931524148
R2 score (Test) 0.6110160765038436
R2 score (Train) 0.9498467843280824


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


*Bagging and Pasting*

In [55]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=10),
                               n_estimators=100,
                               max_samples=376,
                                random_state=42)#take 80% of the total samples

bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.10437617101043273
RMSE 0.19848549053799866
R2 score 0.6348792593513751


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


For target with LTIR*3 and features with the rest
max depth =10
n=100
max samples =376

MAE 0.10437617101043273
RMSE 0.19848549053799866
R2 score 0.6348792593513751
Good scores

*Gradient Boosting*

In [57]:
print(y_train.shape)  # Should print (376,) not (376, 3)


(376, 3)


In [60]:
y_train_gb = y_train["LTIR total"] 
y_test_gb = y_test["LTIR total"]
gb_reg = GradientBoostingRegressor(max_depth=10,
                                   n_estimators=100,
                                   random_state=42)
gb_reg.fit(X_train, y_train_gb)
pred = gb_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test_gb))
print("RMSE", mean_squared_error(pred, y_test_gb, squared=False))
print("R2 score", gb_reg.score(X_test, y_test_gb))
print("R2 score", gb_reg.score(X_train, y_train_gb))

MAE 0.1117864129037802
RMSE 0.39307952363149157
R2 score -0.1412316209504969
R2 score 0.999999999294211


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


GB results (10,100,42):
MAE 0.1117864129037802
RMSE 0.39307952363149157
R2 score -0.1412316209504969
R2 score 0.999999999294211
Small R2

*Adaptive Boosting*

In [62]:
y_train_ab = y_train["LTIR total"] 
y_test_ab = y_test["LTIR total"]
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=100,
                            random_state=42)
ada_reg.fit(X_train, y_train_ab)
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test_ab))
print("RMSE", mean_squared_error(pred, y_test_ab, squared=False))
print("R2 score", ada_reg.score(X_test, y_test_ab))
print("R2 score", ada_reg.score(X_train, y_train_ab))

MAE 0.04495567375886524
RMSE 0.18898996261512713
R2 score 0.7361904989257821
R2 score 0.9989796794681831


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ab results (10,100,42):
MAE 0.04495567375886524
RMSE 0.18898996261512713
R2 score 0.7361904989257821
R2 score 0.9989796794681831
Best score

Logistic regression ?
Sarimax = exogenous feature ? (Used for air pollution prediction) without it's Sarima

*SARIMA*

*Logistic regression*

**Overfitting?**

**Cross validation**

**Using the model for prediction**

In [52]:
future_years = np.array(range(2025, 2031)).reshape(-1, 1)
future_df = pd.DataFrame({"Year": future_years.flatten()})

# Step 1: Create lag and rolling mean features for future data
for col in ["FAR total", "TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"]:
    future_df[f"{col}_lag1"] = np.nan  # Placeholder for lag1
    future_df[f"{col}_lag2"] = np.nan  # Placeholder for lag2
    future_df[f"{col}_rolling_mean"] = np.nan  # Placeholder for rolling mean



In [61]:
# Step 2: One-hot encode the 'Company code' for future data
companies = df['Company code'].unique()  # Get unique company codes used in training
company_cols = [f'Company_{company}' for company in companies]  # List of one-hot encoded company columns

company_cols 

['Company_E         ',
 'Company_F         ',
 'Company_G         ',
 'Company_H         ',
 'Company_I         ',
 'Company_J         ',
 'Company_K         ',
 'Company_L         ',
 'Company_M         ',
 'Company_N         ',
 'Company_O         ',
 'Company_P         ',
 'Company_Q         ',
 'Company_R         ',
 'Company_S         ',
 'Company_T         ',
 'Company_V         ',
 'Company_W         ',
 'Company_X         ',
 'Company_Y         ',
 'Company_Overall   ',
 'Company_AA        ',
 'Company_BB        ',
 'Company_CC        ',
 'Company_DD        ',
 'Company_EE        ',
 'Company_FF        ',
 'Company_GG        ',
 'Company_HH        ',
 'Company_II        ',
 'Company_KK        ',
 'Company_LL        ',
 'Company_MM        ',
 'Company_NN        ',
 'Company_OO        ',
 'Company_PP        ',
 'Company_QQ        ',
 'Company_RR        ',
 'Company_SS        ',
 'Company_TT        ',
 'Company_UU        ',
 'Company_VV        ',
 'Company_WW        ',
 'Company_A

In [58]:
# Initialize with zeros for future_df (since future_df doesn't have companies yet, assume 0s)
for col in company_cols:
    future_df[col] = 0

# Step 3: Add lag and rolling mean data from the most recent year in the historical data
# We'll take the last row of your historical data (i.e., the latest year) and use it for future data as placeholders.
latest_data = model_df.iloc[-1]

# Loop through each feature to assign the latest available data for the future years
for col in ["FAR total", "TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"]:
    future_df[f"{col}_lag1"] = latest_data[col]  # Assign latest value as lag1 for future
    future_df[f"{col}_lag2"] = latest_data[col]  # Assign latest value as lag2 for future
    future_df[f"{col}_rolling_mean"] = latest_data[col]  # Use latest value as rolling mean placeholder



In [59]:
# Step 4: Align features for future data (same as used in training)
# Assuming the model was trained with the same set of features, align these columns for prediction.
future_df_with_features = future_df[["Year"] + company_cols + 
                                     ["FAR total_lag1", "FAR total_lag2", "FAR total_rolling_mean", 
                                      "TRIR total_lag1", "TRIR total_lag2", "TRIR total_rolling_mean", 
                                      "LTIR total_lag1", "LTIR total_lag2", "LTIR total_rolling_mean"]]



In [60]:
# Step 5: Use the trained model to predict future values
# Assuming that your model has already been trained, use it to predict.
future_predictions = model.predict(future_df_with_features)

# Step 6: Store predictions in the dataframe
future_df[["TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"]] = future_predictions

# Step 7: Print out the future predictions dataframe
print(future_df)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Company_00        
- Company_A         
- Company_AA        
- Company_AAA       
- Company_B         
- ...
Feature names seen at fit time, yet now missing:
- Company code_A         
- Company code_AA        
- Company code_AAA       
- Company code_B         
- Company code_BB        
- ...


In [22]:
for col in ["FAR total", "TRIR total", "TRIR company only","TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only" ]:
    df[f"{col}_rolling_mean"] = df[col].rolling(window=3).mean()
df.dropna(inplace=True)  # Drop NaN values created by shifting

In [23]:
df = pd.get_dummies(df, columns=["Company code"], drop_first=True)

Sans FAR

In [32]:
from sklearn.model_selection import train_test_split

# Define features (X) and targets (y)
X = df.drop(columns=["FAR total", "TRIR total", "TRIR company only","TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"])  # Features
y = df[[ "TRIR total", "TRIR company only","TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"]]  # Target variables

# Split the dataset (train on past data, test on future data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Initialize model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 0.3491333333333333


In [35]:
import numpy as np
import pandas as pd

# Step 1: Prepare the future years (2025-2030)
future_years = np.array(range(2025, 2031)).reshape(-1, 1)  # Example: Predict for 2025-2030

# Create a DataFrame for future years
future_df = pd.DataFrame({"Year": future_years.flatten()})

# Step 2: One-hot encode the company column for the future data (like you did during training)
# Assuming df has the 'Company code' column from the original dataset
companies = df['Company code'].unique()  # Get unique company codes used in training

# Make sure that 'Company code' column exists in the future_df (otherwise, add it)
# Add a placeholder company data for future predictions (you can modify this as needed)
future_df['Company code'] = companies[0]  # This is just an example, adjust as needed

# One-hot encode the 'Company code' column (future_df should have the same structure as training data)
future_companies_df = pd.get_dummies(future_df['Company code'], prefix='Company', columns=companies)

# Add missing columns for companies not in future data
for company in companies:
    if f'Company_{company}' not in future_companies_df.columns:
        future_companies_df[f'Company_{company}'] = 0

# Concatenate one-hot encoded columns to future_df
future_df = pd.concat([future_df, future_companies_df], axis=1)

# Step 3: Use the trained model to predict future values
# Assuming the model is already trained, you can now make predictions on future_df
future_predictions = model.predict(future_df)

# Step 4: Store predictions in the dataframe
future_df[[ "TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only"]] = future_predictions

# Step 5: Print out the future predictions dataframe
print(future_df)


KeyError: 'Company code'